In [157]:
import libsumo as traci
import time
import random
import numpy as np
import csv
import os
import random, numpy as np
import pandas as pd, matplotlib.pyplot as plt

SEED = 8
random.seed(SEED)
np.random.seed(SEED)
# Imports and configuration
# Use sumo or sumo-gui depending on your preference
SUMO_GUI_BINARY  = "sumo-gui"  # or "sumo-gui"
SUMO_BINARY  = "sumo"  # or "sumo-gui"
SUMO_CFG  = "./sumo_cfg/simulation.sumocfg"  # your .sumocfg file
SUMO_STATE = "./sumo_cfg/simulation_state.xml"  # your initial state file
LOG_DIR= "logs"

# Global timing constants
REDUCE_AVG_WAIT_TIME_W, FAIRNESS_W = 0.7, 0.3 # prioritize reduce wait time
CYCLE_LENGTH_DEFAULT = 90   # s
MAX_CYCLE_LENGTH = 120      # s
MIN_CYCLE_LENGTH = 60      # s
LOST_TIME = 12              # s (amber + all-red total)
GREEN_MIN = 15             # s per direction
MIN_GREEN_SPLIT = GREEN_MIN / (MIN_CYCLE_LENGTH-LOST_TIME)  # minimum green split ratio
END_TIME = 16200

In [158]:
#Intersection metadata
# Define lane groups for metric aggregation
NS_LANES = ["1200728225#1_0", "1200728225#1_1", "1221994726#0_0", "1221994726#0_1"]  # replace with your real lane IDs
EW_LANES = ["1265822568#3_0", "1265822568#3_1", "1265822568#3_2"]

# Your target traffic light ID
TL_ID = "cluster_13075564400_13075589603_411926344"  # replace with your actual traffic light id in SUMO net

## Simulation Section

In [168]:
def find_avg_halt_range(ns_lanes, ew_lanes):
    ns_delay_proxy = ew_delay_proxy = 0
    max_avg_halt = 0
    min_avg_halt = 99999
    while traci.simulation.getTime() < END_TIME:
        ns_delay_proxy = 0
        ew_delay_proxy = 0
        for _ in range(CYCLE_LENGTH_DEFAULT):
            traci.simulationStep()
            signal_state = traci.trafficlight.getRedYellowGreenState(TL_ID)

            # Split signal state: first 5 (EW), last 7 (NS)
            ew_state = signal_state[:5]
            ns_state = signal_state[5:]

            # Detect if direction is in red
            ew_red = "r" in ew_state
            ns_red = "r" in ns_state
            # Accumulate waiting times and vehicles only when direction is red
            if ns_red:
                ns_delay_proxy += sum(traci.lane.getLastStepHaltingNumber(l) for l in ns_lanes)
            if ew_red:
                ew_delay_proxy += sum(traci.lane.getLastStepHaltingNumber(l) for l in ew_lanes)
                
            # Total average queue length
            total_avg_halt = (ns_delay_proxy + ew_delay_proxy)
        if(total_avg_halt > max_avg_halt):
            max_avg_halt = total_avg_halt
        if(total_avg_halt < min_avg_halt):
            min_avg_halt = total_avg_halt
    print(f"❌ Stopping at END_TIME.")
    usable_time = CYCLE_LENGTH_DEFAULT - LOST_TIME
    
    return max_avg_halt/usable_time,min_avg_halt/usable_time

In [167]:
sumoCmd = [
    SUMO_BINARY,
    "-c", SUMO_CFG,
    "--seed", str(SEED),
]
traci.start(sumoCmd)
print(traci.trafficlight.getControlledLanes(TL_ID))
traci.simulationStep()
signal_state = traci.trafficlight.getRedYellowGreenState(TL_ID)
print(signal_state)

(max_avg_halt,min_avg_halt) = find_avg_halt_range(NS_LANES, EW_LANES)
print('Max avg halt', max_avg_halt)
print('Min avg halt', min_avg_halt)
traci.close()

('1265822568#3_0', '1265822568#3_0', '1265822568#3_1', '1265822568#3_2', '1265822568#3_2', '1200728225#1_0', '1200728225#1_1', '1200728225#1_1', '1200728225#1_1', '1221994726#0_0', '1221994726#0_0', '1221994726#0_1')
rrrrrGGggGGG
❌ Stopping at END_TIME.
Max avg halt 29.0
Min avg halt 3.0641025641025643


In [173]:
#Delay rate (queue area per usable second)
MIN_DELAY_RATE_PER_CYCLE = 2
MAX_DELAY_RATE_PER_CYCLE = 40

In [174]:
def apply_plan(tl_id, g_main, g_cross, amber=3, all_red=3):
    """
    Define a 2-phase signal plan (NS and EW) with amber and all-red times.
    """
    amrber_red_phase_duration = amber + all_red
    phases = [
        traci.trafficlight.Phase(g_main, "rrrrrGGggGGG"),   # NS green
        traci.trafficlight.Phase(amrber_red_phase_duration, "rrrrryyyyyyy"),    # NS amber
        traci.trafficlight.Phase(g_cross, "GGGGGrrrrrrr"),  # EW green
        traci.trafficlight.Phase(amrber_red_phase_duration, "yyyyyrrrrrrr"),    # EW amber
    ]

    logic = traci.trafficlight.Logic("custom_logic", 0, 0, phases)
    traci.trafficlight.setCompleteRedYellowGreenDefinition(tl_id, logic)

    # Metrics calculation
def cycle_metrics(ns_lanes, ew_lanes, steps):
    ns_delay_proxy = ew_delay_proxy = 0

    for _ in range(steps):
        traci.simulationStep()
        signal_state = traci.trafficlight.getRedYellowGreenState(TL_ID)

         # Split signal state: first 5 (EW), last 7 (NS)
        ew_state = signal_state[:5]
        ns_state = signal_state[5:]

        # Detect if direction is in red
        ew_red = "r" in ew_state
        ns_red = "r" in ns_state
        # Accumulate waiting times and vehicles only when direction is red
        if ns_red:
            ns_delay_proxy += sum(traci.lane.getLastStepHaltingNumber(l) for l in ns_lanes)
        if ew_red:
            ew_delay_proxy += sum(traci.lane.getLastStepHaltingNumber(l) for l in ew_lanes)
            
    # Fairness (bounded 0–1)
    O2_norm = abs(ns_delay_proxy - ew_delay_proxy) / (ns_delay_proxy + ew_delay_proxy + 1e-5)
    # Total average queue length
    delay_rate = (ns_delay_proxy + ew_delay_proxy)/(steps-LOST_TIME)
    delay_rate_norm = (delay_rate - MIN_DELAY_RATE_PER_CYCLE) / (MAX_DELAY_RATE_PER_CYCLE - MIN_DELAY_RATE_PER_CYCLE)
    O1_norm = max(0.0, min(delay_rate_norm, 1.0))

    return O1_norm, O2_norm

def get_green_split(s, C):
    g_main = max(GREEN_MIN, round((C - LOST_TIME) * s))
    g_cross = max(GREEN_MIN, (C - LOST_TIME) - g_main)
    return g_main, g_cross


## Evaluate Section

In [175]:
# Evaluation function
def evaluate(s, C, isReset=True):
    try:    
        # Restore identical starting conditions before evaluating this candidate
        if isReset:
            traci.simulation.loadState(SUMO_STATE)
            time.sleep(0.01)
        g_main, g_cross = get_green_split(s, C)
        apply_plan(TL_ID, g_main, g_cross)
        steps = int(C)
        O1_norm, O2_norm = cycle_metrics(NS_LANES, EW_LANES, steps)

    except traci.exceptions.FatalTraCIError as e:
        print("⚠️ SUMO crashed during evaluation:", e)
        return C, 1  # return a default
    return O1_norm, O2_norm

def score_function(O1, O2):
    score = (REDUCE_AVG_WAIT_TIME_W * O1) + (FAIRNESS_W * O2)
    return score

# x consist of split-Cycle
def evaluate_candidate(x):
    s, C = float(x[0]), round(float(x[1]))  # round C for realism
    O1, O2 = evaluate(s, C)
    return O1, O2

def robust_evaluate(x, n_cycles=2, λ=0.3):
    """Returns (robust_score, mean_O1, mean_O2)"""
    s, C = float(x[0]), round(float(x[1]))
    traci.simulation.loadState(SUMO_STATE)
    scores = []
    O1s = []
    O2s = []
    for _ in range(n_cycles):
        O1, O2 = evaluate(s, C, isReset=False)  # continue simulation
        O1s.append(O1)
        O2s.append(O2)
        scores.append(score_function(O1, O2))
    robust_score = np.mean(scores) + λ * np.std(scores)

    return robust_score, np.mean(O1s), np.mean(O2s)

In [176]:
from dataclasses import dataclass

@dataclass
class DEResult:
    s: float
    C: float
    O1: float
    O2: float
    score: float
    robust_score: float
    elapsed: float
    gen_history: list

## Evolution Algorithm Section

In [177]:
 # Include previous elite as first individual (temporal continuity)
def init_population(bounds, pop_size, elite_last=None):
    pop = [np.array([random.uniform(*b) for b in bounds]) for _ in range(pop_size)]
    if elite_last is not None:
        pop[0] = np.array([elite_last[0], elite_last[1]])  # temporal continuity
    return pop

# Early stopping check using std
def early_stop_check(gen_history, patience, min_delta, elapsed, time_budget_s):
    if len(gen_history) > patience:
        recent_scores = [h[-1] for h in gen_history[-patience:]]
        score_std = np.std(recent_scores)
        if score_std < min_delta:
            print(f"Early stopping: score std {score_std:.6f} < {min_delta}")
            return True
        
    if elapsed >= time_budget_s:
            return True
    return False

 # --- Select top-K and re-evaluate robustly
def final_robust_selection(pop, scores, K_ratio=0.5):
    K = round(len(pop) * K_ratio)
    ranked = sorted(
        [(pop[i], scores[i], score_function(*scores[i])) for i in range(len(pop))],
        key=lambda x: x[2]
    )
    topK = ranked[:K]
    evaluated = [(x, *robust_evaluate(x)) for x, _, _ in topK]
    winner, best_robust_score, O1_best, O2_best = min(evaluated, key=lambda z: z[1])
    s_best, C_best = winner
    return s_best, C_best, O1_best, O2_best, best_robust_score

def evolve_generation(pop, scores, bounds, F, CR):
    scale = np.array([1.0, 1.0 / (bounds[1][1] - bounds[1][0])])  # scale by range
    pop_size = len(pop)
    new_pop, new_scores = pop.copy(), scores.copy()

    for i in range(pop_size):
        idxs = list(range(pop_size))
        idxs.remove(i)
        r1, r2, r3 = random.sample(idxs, 3)

        # Mutation
        mutant = pop[r1] + F * scale * (pop[r2] - pop[r3])
        mutant = np.clip(mutant, [b[0] for b in bounds], [b[1] for b in bounds])

        # Crossover (ensure at least one mutant gene)
        trial = np.copy(pop[i])
        jrand = random.randrange(len(bounds))
        for j in range(len(bounds)):
            if j == jrand or random.random() < CR:
                trial[j] = mutant[j]

        # Evaluate and selection
        O1_t, O2_t = evaluate_candidate(trial)
        O1_i, O2_i = scores[i]
        score_t = score_function(O1_t, O2_t)
        score_i = score_function(O1_i, O2_i)

        if score_t <= score_i:
            new_pop[i], new_scores[i] = trial, (O1_t, O2_t)

    return new_pop, new_scores

def differential_evolution(
    elite_last=None,
    time_budget_s=120,
    pop_size=16,
    F=0.8,
    CR=0.6,
    patience=5,
    min_delta=1e-3,
):
    """
    Differential Evolution (DE/rand/1/bin)
    for 2 parameters: (s, C)
    Optimizes two objectives (O1, O2) within a strict time budget.

    Returns: DEResult(s_best, C_best, O1_best, O2_best, score_best, best_robust_score, elapsed)
    """

    start = time.perf_counter()
    bounds = [(MIN_GREEN_SPLIT, 1-MIN_GREEN_SPLIT), (MIN_CYCLE_LENGTH, MAX_CYCLE_LENGTH)]  # (s_min, s_max), (C_min, C_max)

    pop = init_population(bounds, pop_size, elite_last)
    scores = [None] * pop_size

    # --- Evaluate initial population
    for i in range(pop_size):
        O1, O2 = evaluate_candidate(pop[i])
        scores[i] = (O1, O2)

    gen = 0
    gen_history = []  # store per-generation bests
    # --- DE optimization loop (time-bounded)
    while time.perf_counter() - start < time_budget_s:
        gen += 1
        pop, scores = evolve_generation(pop, scores, bounds, F, CR)

       # Summary
        best_idx = min(range(pop_size),
                       key=lambda k: score_function(scores[k][0], scores[k][1]))
        best = pop[best_idx]
        s_best, C_best = best
        O1_best, O2_best = scores[best_idx]
        score_best = score_function(O1_best, O2_best)
        elapsed = time.perf_counter() - start

        # print(f"gen {gen:02d} | t={elapsed:4.1f}s | "
        #       f"s={s_best:.3f} C={C_best:.1f} "
        #       f"O1={O1_best:.3f} O2={O2_best:.3f} score={score_best:.3f}")

        gen_history.append((elapsed, s_best, C_best, O1_best, O2_best, score_best))

        if early_stop_check(gen_history, patience, min_delta, elapsed, time_budget_s):
            break

    # --- Final best with robust evaluation
    s_best, C_best, O1_best, O2_best, best_robust_score = final_robust_selection(pop, scores)
    elapsed = time.perf_counter() - start

    return DEResult(s_best, C_best, O1_best, O2_best, score_best, best_robust_score, elapsed, gen_history)


In [178]:
import csv, os
from datetime import datetime

def log_cycle_result(
    cycle,
    s,
    C,
    O1,
    O2,
    score_best,
    robust_score,
    elapsed_s,
    suffix="",
    out_dir=LOG_DIR,
    prefix="DE_cycle",
):
    """
    Append one cycle results to a CSV log file
    """
    # --- Create output folder
    os.makedirs(out_dir, exist_ok=True)

    # Main summary file (aggregated cycle-level results)
    summary_file = os.path.join(out_dir, f"{prefix}_summary_{suffix}.csv")
    write_header = not os.path.exists(summary_file)

    with open(summary_file, "a", newline="") as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow([
                "cycle", "s", "C", "O1", "O2",
                "score_best", "robust_score", "elapsed_s"
            ])
        writer.writerow([
            cycle, round(s, 3), round(C, 1), round(O1, 3), round(O2, 3),
            round(score_best, 4), round(robust_score, 4), round(elapsed_s, 2)
        ])

    return summary_file

In [179]:
def seed_de_simulation():
    # Start SUMO
    sumoCmd = [
        SUMO_BINARY,
        "-c", SUMO_CFG,
        "--seed", str(SEED),
    ]
    log_suffix = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = ""

    elite = [0.5, CYCLE_LENGTH_DEFAULT]  # initial elite

    traci.start(sumoCmd)
    for cycle in range(300):  # min 50s per cycle ~ 325 cycle for 16200s
        time_pased = traci.simulation.getTime()
        print(f"\nTime passed at cycle {cycle+1}: {time_pased}s.")
        if time_pased >= END_TIME:
            print(f"❌ No more vehicles at cycle {cycle+1}, stopping early.")
            break
        print(f"=== Cycle {cycle+1} (t={time_pased:.1f}s) ===")
        #  Save the current live SUMO state once
        traci.simulation.saveState(SUMO_STATE)

        #  Run DE using that snapshot as baseline
        result = differential_evolution(elite_last=elite)
        s, C, O1, O2, score_best, robust_score, elapsed_s, gen_history = (
            result.s, result.C, result.O1, result.O2,
            result.score, result.robust_score, result.elapsed, result.gen_history
        )
        elite = (s, C)

        #  Restore pre-optimization state before applying best plan
        traci.simulation.loadState(SUMO_STATE)

        #  Apply only the *best plan* to the live SUMO world
        g_main, g_cross = get_green_split(s, C)
        apply_plan(TL_ID, g_main, g_cross)

        for _ in range(int(C)):
            traci.simulationStep()
            
        #  Log and write results
        print(
            f"Chosen split={s:.2f}, C={C:.1f}, "
            f"O1={O1:.3f}, O2={O2:.3f}, score={score_best:.3f}, time={elapsed_s:.2f}s, gen={len(gen_history)}"
        )
        # Save logs
        log_file = log_cycle_result(
            cycle + 1, s, C, O1, O2, score_best, robust_score, elapsed_s,
            suffix=log_suffix, out_dir="logs", prefix="traffic_DE"
        )

    traci.close()    
    print(f"\n✅ Simulation completed — results saved to {log_file}")
890

# Run the function
seed_de_simulation()


Early stopping: score std 0.000000 < 0.001
Chosen split=0.69, C=103.0, O1=0.698, O2=0.080, score=0.471, time=68.58s, gen=6

Time passed at cycle 70: 6161.0s.
=== Cycle 70 (t=6161.0s) ===
Early stopping: score std 0.000000 < 0.001
Chosen split=0.66, C=95.3, O1=0.705, O2=0.158, score=0.495, time=75.92s, gen=7

Time passed at cycle 71: 6256.0s.
=== Cycle 71 (t=6256.0s) ===
Early stopping: score std 0.000000 < 0.001
Chosen split=0.69, C=103.1, O1=0.729, O2=0.161, score=0.440, time=76.38s, gen=7

Time passed at cycle 72: 6359.0s.
=== Cycle 72 (t=6359.0s) ===
Early stopping: score std 0.000000 < 0.001
Chosen split=0.58, C=114.1, O1=0.717, O2=0.111, score=0.492, time=127.11s, gen=12

Time passed at cycle 73: 6473.0s.
=== Cycle 73 (t=6473.0s) ===
Early stopping: score std 0.000000 < 0.001
Chosen split=0.69, C=113.8, O1=0.618, O2=0.159, score=0.433, time=75.92s, gen=6

Time passed at cycle 74: 6586.0s.
=== Cycle 74 (t=6586.0s) ===
Early stopping: score std 0.000000 < 0.001
Chosen split=0.66, C=

Early stopping: score std 0.000000 < 0.001


Chosen split=0.61, C=74.9, O1=0.637, O2=0.136, score=0.368, time=86.52s, gen=7

Time passed at cycle 79: 7031.0s.
=== Cycle 79 (t=7031.0s) ===
Early stopping: score std 0.000967 < 0.001
Chosen split=0.68, C=75.9, O1=0.606, O2=0.105, score=0.465, time=107.40s, gen=9

Time passed at cycle 80: 7106.0s.
=== Cycle 80 (t=7106.0s) ===


Early stopping: score std 0.000000 < 0.001
Chosen split=0.69, C=86.1, O1=0.527, O2=0.053, score=0.333, time=112.36s, gen=9

Time passed at cycle 81: 7192.0s.
=== Cycle 81 (t=7192.0s) ===
Early stopping: score std 0.000000 < 0.001
Chosen split=0.69, C=73.9, O1=0.529, O2=0.090, score=0.305, time=79.71s, gen=6

Time passed at cycle 82: 7265.0s.
=== Cycle 82 (t=7265.0s) ===
Early stopping: score std 0.000000 < 0.001
Chosen split=0.66, C=73.5, O1=0.639, O2=0.045, score=0.357, time=119.65s, gen=10

Time passed at cycle 83: 7338.0s.
=== Cycle 83 (t=7338.0s) ===
Early stopping: score std 0.000000 < 0.001
Chosen split=0.66, C=85.6, O1=0.605, O2=0.030, score=0.432, time=85.26s, gen=6

Time passed at cycle 84: 7423.0s.
=== Cycle 84 (t=7423.0s) ===
Chosen split=0.69, C=86.1, O1=0.766, O2=0.126, score=0.513, time=133.07s, gen=10

Time passed at cycle 85: 7509.0s.
=== Cycle 85 (t=7509.0s) ===
Early stopping: score std 0.000000 < 0.001
Chosen split=0.67, C=88.2, O1=0.616, O2=0.026, score=0.467, time=

Chosen split=0.69, C=108.7, O1=0.715, O2=0.051, score=0.498, time=107.01s, gen=6

Time passed at cycle 95: 8340.0s.
=== Cycle 95 (t=8340.0s) ===
Early stopping: score std 0.000000 < 0.001


...

Chosen split=0.65, C=61.4, O1=0.568, O2=0.051, score=0.371, time=138.85s, gen=5

Time passed at cycle 130: 11539.0s.
=== Cycle 130 (t=11539.0s) ===
Chosen split=0.69, C=86.4, O1=0.555, O2=0.045, score=0.426, time=139.69s, gen=5

Time passed at cycle 131: 11625.0s.
=== Cycle 131 (t=11625.0s) ===
Chosen split=0.68, C=85.8, O1=0.634, O2=0.056, score=0.407, time=139.95s, gen=5

Time passed at cycle 132: 11710.0s.
=== Cycle 132 (t=11710.0s) ===
Chosen split=0.69, C=113.2, O1=0.635, O2=0.077, score=0.490, time=146.45s, gen=5

Time passed at cycle 133: 11823.0s.
=== Cycle 133 (t=11823.0s) ===
Chosen split=0.69, C=84.1, O1=0.643, O2=0.146, score=0.434, time=155.12s, gen=5

Time passed at cycle 134: 11907.0s.
=== Cycle 134 (t=11907.0s) ===
Chosen split=0.69, C=83.7, O1=0.684, O2=0.181, score=0.385, time=143.41s, gen=5

Time passed at cycle 135: 11990.0s.
=== Cycle 135 (t=11990.0s) ===
Chosen split=0.67, C=114.7, O1=0.782, O2=0.161, score=0.543, time=144.16s, gen=5

Time passed at cycle 136: 121

Chosen split=0.69, C=107.5, O1=0.767, O2=0.078, score=0.556, time=155.03s, gen=5

Time passed at cycle 147: 13150.0s.
=== Cycle 147 (t=13150.0s) ===
Chosen split=0.63, C=113.7, O1=0.703, O2=0.011, score=0.445, time=140.59s, gen=4

Time passed at cycle 148: 13263.0s.
=== Cycle 148 (t=13263.0s) ===
Chosen split=0.67, C=85.9, O1=0.673, O2=0.146, score=0.483, time=135.90s, gen=4

Time passed at cycle 149: 13348.0s.
=== Cycle 149 (t=13348.0s) ===
Chosen split=0.55, C=75.5, O1=0.644, O2=0.019, score=0.480, time=154.63s, gen=5

Time passed at cycle 150: 13423.0s.
=== Cycle 150 (t=13423.0s) ===
Chosen split=0.69, C=94.3, O1=0.668, O2=0.090, score=0.389, time=160.13s, gen=5

Time passed at cycle 151: 13517.0s.
=== Cycle 151 (t=13517.0s) ===
Chosen split=0.63, C=84.7, O1=0.763, O2=0.152, score=0.584, time=137.06s, gen=4

Time passed at cycle 152: 13601.0s.
=== Cycle 152 (t=13601.0s) ===
Chosen split=0.69, C=114.6, O1=0.713, O2=0.116, score=0.500, time=136.75s, gen=4

Time passed at cycle 153: 13

Chosen split=0.69, C=88.9, O1=0.705, O2=0.042, score=0.459, time=155.15s, gen=4

Time passed at cycle 174: 15545.0s.
=== Cycle 174 (t=15545.0s) ===


Chosen split=0.68, C=108.5, O1=0.774, O2=0.096, score=0.489, time=152.81s, gen=4

Time passed at cycle 175: 15653.0s.
=== Cycle 175 (t=15653.0s) ===
Chosen split=0.65, C=82.8, O1=0.652, O2=0.093, score=0.384, time=149.58s, gen=4

Time passed at cycle 176: 15735.0s.
=== Cycle 176 (t=15735.0s) ===
Chosen split=0.65, C=80.3, O1=0.664, O2=0.143, score=0.459, time=150.65s, gen=4

Time passed at cycle 177: 15815.0s.
=== Cycle 177 (t=15815.0s) ===
Chosen split=0.65, C=116.7, O1=0.679, O2=0.144, score=0.429, time=153.12s, gen=4

Time passed at cycle 178: 15931.0s.
=== Cycle 178 (t=15931.0s) ===
Chosen split=0.68, C=97.6, O1=0.629, O2=0.142, score=0.425, time=152.24s, gen=4

Time passed at cycle 179: 16028.0s.
=== Cycle 179 (t=16028.0s) ===


Chosen split=0.64, C=96.8, O1=0.736, O2=0.047, score=0.476, time=159.55s, gen=4

Time passed at cycle 180: 16124.0s.
=== Cycle 180 (t=16124.0s) ===
Chosen split=0.69, C=112.1, O1=0.620, O2=0.100, score=0.495, time=151.04s, gen=4

Time passed at cycle 181: 16236.0s.
❌ No more vehicles at cycle 181, stopping early.

✅ Simulation completed — results saved to logs/traffic_DE_summary_20251106_225623.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def visualize_pow_results(log_dir, suffix="", prefix="traffic_DE"):
    """
    Visualize core PoW evidence from your DE summary CSV.
    Expects: CSV created by log_cycle_result()
    Plots:
        1. Cycle length evolution (adaptivity)
        2. Objective trends (O1, O2)
        3. Score trend (convergence)
        4. Optimization runtime per cycle
        5. Baseline vs. EA improvement summary (optional if baseline exists)
    """
    # --- Load summary CSV
    summary_file = os.path.join(log_dir, f"{prefix}_summary_{suffix}.csv")
    if not os.path.exists(summary_file):
        print(f"❌ Summary file not found: {summary_file}")
        return

    df = pd.read_csv(summary_file)
    print(f"Loaded {len(df)} cycles from {summary_file}")

    # --- 1. Cycle length evolution
    plt.figure(figsize=(7,4))
    plt.plot(df["cycle"], df["C"], marker='o')
    plt.xlabel("Cycle #")
    plt.ylabel("Cycle length (s)")
    plt.title("Adaptive cycle length over time")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # --- 2. Objective trends
    plt.figure(figsize=(7,4))
    plt.plot(df["cycle"], df["O1"], label="Avg delay (O1)")
    plt.plot(df["cycle"], df["O2"], label="Fairness (O2)")
    plt.xlabel("Cycle #")
    plt.ylabel("Objective value")
    plt.title("Objective trends over cycles")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # --- 3. Score trend
    plt.figure(figsize=(7,4))
    plt.plot(df["cycle"], df["score_best"], label="Score (per cycle)", color='orange')
    plt.plot(df["cycle"], df["robust_score"], label="Robust score (multi-cycle)", color='green')
    plt.xlabel("Cycle #")
    plt.ylabel("Score")
    plt.title("Optimization score per cycle")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # --- 4. Runtime feasibility
    plt.figure(figsize=(7,4))
    plt.plot(df["cycle"], df["elapsed_s"], label="Runtime per optimization", color='red')
    plt.axhline(y=15, color='gray', linestyle='--', label="15s real-time bound")
    plt.xlabel("Cycle #")
    plt.ylabel("Computation time (s)")
    plt.title("Optimization runtime per cycle")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # --- 5. Summary statistics
    print("\n=== Summary statistics ===")
    print(df[["C","O1","O2","score_best","elapsed_s"]].describe().round(3))
    avg_runtime = df["elapsed_s"].mean()
    print(f"\n✅  Average runtime per DE optimization: {avg_runtime:.2f}s")
    print("✅ Visualization complete — core PoW evidence generated.")


In [ ]:
visualize_pow_results(log_dir=LOG_DIR, suffix="20251105_203454")

❌ Summary file not found: logs/traffic_DE_summary_20251105_203454.csv


In [ ]:
def baseline_simulation(fixed_s=0.5, fixed_C=90, total_cycles=300):
    """
    Baseline simulation using a fixed signal plan (no optimization).
    Uses identical SUMO setup and logging as the DE version
    for direct comparison.
    """
    sumoCmd = [
        SUMO_BINARY,
        "-c", SUMO_CFG,
        "--seed", str(SEED),
    ]
    log_suffix = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = ""

    print(f"Running baseline simulation: s={fixed_s}, C={fixed_C}s for {total_cycles} cycles")

    traci.start(sumoCmd) 
    for cycle in range(total_cycles):
        time_pased = traci.simulation.getTime()
        print(f"Time passed at cycle {cycle+1}: {time_pased}s.")
        if time_pased >= END_TIME:
            print(f"❌ No more vehicles at cycle {cycle+1}, stopping early.")
            break
        print(f"\n=== Cycle {cycle+1} (t={time_pased:.1f}s) ===")
        
        # Apply fixed plan
        g_main, g_cross = get_green_split(fixed_s, fixed_C)
        apply_plan(TL_ID, g_main, g_cross)

        # Evaluate performance under fixed plan
        O1, O2 = cycle_metrics(NS_LANES, EW_LANES, int(fixed_C))
        score_best = score_function(O1, O2)
        robust_score = 0  # no multi-cycle robust re-evaluation
        elapsed_s = 0.0  # not optimized, so no runtime cost

        # Save current state (optional, just for consistency)
        traci.simulation.saveState(SUMO_STATE)

        # Log
        print(
            f"Fixed split={fixed_s:.2f}, C={fixed_C:.1f}, "
            f"O1={O1:.3f}, O2={O2:.3f}, score={score_best:.3f}"
        )

        # Write results
        log_file = log_cycle_result(
            cycle + 1,
            fixed_s,
            fixed_C,
            O1,
            O2,
            score_best,
            robust_score,
            elapsed_s,
            suffix=log_suffix,
            out_dir="logs",
            prefix="traffic_baseline",
        )
    traci.close()   
    print(f"\n✅ Baseline completed — results saved to {log_file}")

baseline_simulation()


Running baseline simulation: s=0.5, C=90s for 300 cycles
Time passed at cycle 1: 0.0s.

=== Cycle 1 (t=0.0s) ===
Fixed split=0.50, C=90.0, O1=0.734, O2=0.085, score=0.539
Time passed at cycle 2: 90.0s.

=== Cycle 2 (t=90.0s) ===
Fixed split=0.50, C=90.0, O1=1.463, O2=0.595, score=1.202
Time passed at cycle 3: 180.0s.

=== Cycle 3 (t=180.0s) ===
Fixed split=0.50, C=90.0, O1=1.379, O2=0.329, score=1.064
Time passed at cycle 4: 270.0s.

=== Cycle 4 (t=270.0s) ===
Fixed split=0.50, C=90.0, O1=1.197, O2=0.357, score=0.945
Time passed at cycle 5: 360.0s.

=== Cycle 5 (t=360.0s) ===
Fixed split=0.50, C=90.0, O1=1.141, O2=0.062, score=0.817
Time passed at cycle 6: 450.0s.

=== Cycle 6 (t=450.0s) ===
Fixed split=0.50, C=90.0, O1=1.200, O2=0.129, score=0.879
Time passed at cycle 7: 540.0s.

=== Cycle 7 (t=540.0s) ===
Fixed split=0.50, C=90.0, O1=0.681, O2=0.274, score=0.559
Time passed at cycle 8: 630.0s.

=== Cycle 8 (t=630.0s) ===
Fixed split=0.50, C=90.0, O1=1.221, O2=0.005, score=0.856
Time 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def compare_logs_by_time(log_dir=LOG_DIR, baseline_prefix='traffic_baseline', de_prefix ='traffic_DE', baseline_suffix = "", de_suffix = ""):
    """
    Compare DE and baseline log files by plotting s, C, O1, and O2 
    against time lapse (cycle × C).

    Parameters:
        de_path (str): Path to DE log CSV file
        baseline_path (str): Path to baseline log CSV file
    """

    de_path = os.path.join(log_dir, f"{de_prefix}_summary_{de_suffix}.csv")
    baseline_path = os.path.join(log_dir, f"{baseline_prefix}_summary_{baseline_suffix}.csv")
    if not os.path.exists(de_path):
        print(f"❌ Summary file not found: {de_path}")
        return
    if not os.path.exists(baseline_path):
        print(f"❌ Summary file not found: {baseline_path}")
        return
    # Load CSVs
    de = pd.read_csv(de_path)
    baseline = pd.read_csv(baseline_path)

     # Ensure same length / alignment
    min_len = min(len(de), len(baseline))
    de = de.head(min_len)
    baseline = baseline.head(min_len)

     # Variables to compare
    vars_to_plot = ["s", "C", "O1", "O2", "score_best"]

    # Plot each variable
    for v in vars_to_plot:
        plt.figure(figsize=(8, 5))
        plt.plot(de["cycle"], de[v], label="DE", linewidth=2)
        plt.plot(baseline["cycle"], baseline[v], label="Baseline", linestyle="--", linewidth=2)
        plt.xlabel("Cycle")
        plt.ylabel(v)
        plt.title(f"Comparison of {v} over Cycle")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

compare_logs_by_time(de_suffix='20251105_203454', baseline_suffix='20251106_084755')

❌ Summary file not found: logs/traffic_DE_summary_20251105_203454.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def compare_score_best(log_dir=LOG_DIR, baseline_prefix='traffic_baseline', de_prefix ='traffic_DE', baseline_suffix = "", de_suffix = ""):
    """
    Compare DE vs Baseline based on score_best over cycles.
    Shows a line plot and prints basic improvement summary.
    
    Parameters:
        de_path (str): Path to DE log CSV file
        baseline_path (str): Path to baseline log CSV file
    """
    de_path = os.path.join(log_dir, f"{de_prefix}_summary_{de_suffix}.csv")
    baseline_path = os.path.join(log_dir, f"{baseline_prefix}_summary_{baseline_suffix}.csv")
    if not os.path.exists(de_path):
        print(f"❌ Summary file not found: {de_path}")
        return
    if not os.path.exists(baseline_path):
        print(f"❌ Summary file not found: {baseline_path}")
        return
    # Load CSVs
    de = pd.read_csv(de_path)
    baseline = pd.read_csv(baseline_path)

    # Load the CSVs
    de = pd.read_csv(de_path)
    baseline = pd.read_csv(baseline_path)

    # Ensure same length / alignment
    min_len = min(len(de), len(baseline))
    de = de.head(min_len)
    baseline = baseline.head(min_len)

    # Plot comparison
    plt.figure(figsize=(8, 5))
    plt.plot(de["cycle"], de["score_best"], label="DE", linewidth=2)
    plt.plot(baseline["cycle"], baseline["score_best"], label="Baseline", linestyle="--", linewidth=2)
    plt.xlabel("Cycle")
    plt.ylabel("score_best")
    plt.title("Comparison of score_best: DE vs Baseline")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Print summary stats
    mean_de = de["score_best"].mean()
    mean_base = baseline["score_best"].mean()
    diff = mean_de - mean_base

    print(f"Average DE score_best: {mean_de:.4f}")
    print(f"Average Baseline score_best: {mean_base:.4f}")
    print(f"Δ Improvement: {diff:+.4f} ({(diff / mean_base) * 100:.2f}% change)")

    if diff > 0:
        print("✅ DE improves over Baseline.")
    elif diff < 0:
        print("❌ DE performs worse than Baseline.")
    else:
        print("⚖️ DE and Baseline perform equally.")

compare_score_best(de_suffix='20251105_203454', baseline_suffix='20251106_084755')

❌ Summary file not found: logs/traffic_DE_summary_20251105_203454.csv
